## CSCS530 Winter 2018
#### Complex Systems 530 - Computer Modeling of Complex Systems (Winter 2018)

  * Course ID: CMPLXSYS 530
  * Course Title: Computer Modeling of Complex Systems
  * Term: Winter 2018

In [ ]:
%matplotlib inline

# Imports
import copy
import networkx as nx
import numpy
import matplotlib.pyplot as plt
import pandas


# Import widget methods
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
# Create a random graph
nodes = 100
edges = 2
prob_out = 0.25
g = nx.newman_watts_strogatz_graph(nodes, edges, prob_out)
print((g.number_of_nodes(), g.number_of_edges()))

In [ ]:
# Draw the random graph
g_layout = nx.spring_layout(g, iterations=100)
nx.draw_networkx(g, pos=g_layout, node_color='white')

In [ ]:
# Pick a random person to infect initially
initial_infected = numpy.random.choice(g.nodes())

# Setup initial S/I/R states
for node_id in g.nodes():
    if node_id == initial_infected:
        g.node[node_id]["state"] = "I"
    else:
        g.node[node_id]["state"] = "S"

In [ ]:
def draw_graph(g, g_layout):
    """
    Draw an SIR visualization.
    """
    # Now we can visualize the infected node's position
    node_map = {"S": [node_id for node_id in g.node if g.node[node_id]["state"] == "S"],
                "I": [node_id for node_id in g.node if g.node[node_id]["state"] == "I"],
                "R": [node_id for node_id in g.node if g.node[node_id]["state"] == "R"],
                }

    # Now we can visualize the infected node's position
    f = plt.figure(figsize=(10,10))
    nx.draw_networkx_nodes(g, g_layout,
                           nodelist=node_map["S"],
                           node_color="white")

    nx.draw_networkx_nodes(g, g_layout,
                           nodelist=node_map["I"],
                           node_color="red")

    nx.draw_networkx_nodes(g, g_layout,
                           nodelist=node_map["R"],
                           node_color="black")

    nx.draw_networkx_edges(g, g_layout, 
                           width=1.0, 
                           alpha=0.5)

    _ = nx.draw_networkx_labels(g, g_layout,
                            dict(zip(g.nodes(), g.nodes())),
                            font_size=10)
    
    ax = f.gca()
    ax.set_aspect(1./ax.get_data_ratio())   

draw_graph(g, g_layout)

In [ ]:
# Probability of infection per edge
prob_infection = 1.0
prob_recovery = 0.5

# Track graph history
g_history = [copy.deepcopy(g)]

# Now run the model 
max_steps = 50
for step in xrange(max_steps):
    # Store changes
    new_infected = []
    new_recovered = []
    
    # Iterate over I and infect any S neighbors
    for node_id in g.nodes():
        if g.node[node_id]["state"] == "I":
            # Infect connected persons with prob_infection rate
            neighbors = g.neighbors(node_id)
            for neighbor_id in neighbors:
                if g.node[neighbor_id]["state"] == "S" \
                    and numpy.random.random() <= prob_infection:
                    new_infected.append(neighbor_id)
            
            # Recover with some rate
            if numpy.random.random() <= prob_recovery:
                new_recovered.append(node_id)
    
    # Update graph
    for node_id in g.nodes():
        if node_id in new_recovered:
            g.node[node_id]["state"] = "R"
        elif node_id in new_infected:
            g.node[node_id]["state"] = "I"

    # Track the latest step
    g_history.append(copy.deepcopy(g))

In [ ]:
def display_graph_step(step=0):
    """
    Display a step from the graph history object.
    """
    draw_graph(g_history[step], g_layout)

interact(display_graph_step,
                step= widgets.IntSlider(min=0, max=len(g_history)-1,
                                     step=1))

Copyright (c) 2014, Michael Bommarito All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

- Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

- Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE